In [27]:
import pandas as pd
import os

In [28]:
def get_meteo_dataframes(prefix):
    lst = []

    for file in os.listdir(path=prefix):
        lst.append(pd.read_csv(os.path.join(prefix, file), sep=';'))
        
    return lst

In [29]:
months_dict = {
    1: "Gennaio",
    2: "Febbraio",
    3: "Marzo",
    4: "Aprile",
    5: "Maggio",
    6: "Giugno",
    7: "Luglio",
    8: "Agosto",
    9: "Settembre",
    10: "Ottobre",
    11: "Novembre",
    12: "Dicembre",
}

In [30]:
def process_dataframes(dfs: list):
    grouped_dfs = []
    
    for df in dfs:
        year = df['DATA'].map(lambda x: x.split('/')[2]).mode().values[0]
        
        df['MESE'] = df['DATA'].map(lambda x: int(x.split('/')[1]))
        
        df = df.rename(columns={'TMEDIA °C': f'TMEDIA °C {year}'}) 
        
        grouped_df = (
            df.groupby(by=['MESE'])[f'TMEDIA °C {year}']
                .mean()
                .round(2)
                .reset_index()
                .sort_values('MESE')
        )
        
        grouped_dfs.append(grouped_df)
        
    return grouped_dfs

In [31]:
df_23 = pd.concat(get_meteo_dataframes("Raw Datasets/Meteo23"))
df_24 = pd.concat(get_meteo_dataframes("Raw Datasets/Meteo24"))

In [32]:
def merge_dataframes(dfs):
    df_final = None

    for i, df in enumerate(dfs):
        if i == 0:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, on='MESE')
            
    return df_final

In [34]:
df = merge_dataframes(process_dataframes([df_23, df_24]))
df['MESE'] = df['MESE'].map(lambda x: months_dict[x])
df

,MESE,TMEDIA °C 2023,TMEDIA °C 2024
0,Gennaio,9.61,10.26
1,Febbraio,9.00,11.34
2,Marzo,12.10,13.10
3,Aprile,13.37,15.40
4,Maggio,18.03,19.52
5,Giugno,23.00,24.97
6,Luglio,28.19,27.90
7,Agosto,25.90,27.45
8,Settembre,23.30,22.53
9,Ottobre,20.35,19.32


In [35]:
df.to_csv('modified_meteo.csv')

Provare bar chart race